# Computation

Computation is a sub-field of computer science that explores types of problems where solution can be created using Computers [[0]](#toa-wikipedia), commonly when a computer comes to mind, its is though of as a physical machine, but it can be anything that can do computation, these were abstract machines before the dawn of modern computing.

A computer is something that can do computations automatically, the theory of automation is called automata.

# Turing Machine

In [ ]:
def contain_all_zero(tape, debug_messages=False):
    # State table.
    table = {
        ('X', '_'): ['_', 'R', 'T'],
        ('X', '0'): ['0', 'R', 'X'],
        ('X', '1'): ['1', 'R', 'Y'],
        ('Y', '_'): ['_', 'R', 'F'],
        ('Y', '0'): ['0', 'R', 'Y'],
        ('Y', '1'): ['1', 'R', 'Y'],
    }

    # Position on tape.
    pos = 0
    # Initial state is first in table.
    state = 'X'

    # Keep going while we are not in a halting state.
    while state not in ['T', 'F']:
        # Print the current status.
        if debug_messages:
            print(''.join(tape[:pos]) + state + ''.join(tape[pos:]))
        # Get the row of the table.
        row = table[(state, tape[pos])]
        # Overwrite the symbol.
        tape[pos] = row[0]
        # Move left or right.
        if row[1] == 'R':
            # Put blanks on tape as necessary.
            if pos == len(tape) - 1:
                tape += ['_']
            # Increase position.
            pos += 1
        else:
            # Put blanks on tape as necessary.
            if pos == 0:
                tape = ['_'] + tape
                # The position on the tape has to move with it.
                pos += 1
            # Decrease position.
            pos -= 1
        # Update the state.
        state = row[2]

    # Print the current status.
    if debug_messages:
        print(''.join(tape[:pos]) + state + ''.join(tape[pos:]))

    return state == 'T'

In [ ]:
# Tape input.
tape = list('0101111')
contain_all_zero(tape, True)

test_cases = [
    (list("0101111"), False),
    (list("1111111"), False),
    (list("1101110"), False),
    (list("0000000"), True)
]

test_pass_count = 0
for test in test_cases:
    print(contain_all_zero(test[0]))
    if contain_all_zero(test[0]) == test[1]:
        test_pass_count += 1

if test_pass_count == len(test_cases):
    print("All tests pass")

The code above is derived from [[2]](#03-computation) Ian McLoughlins, 03-computation.ipynb notebook, I have turned it into a function to allow for reuse, and added some test cases, it currently returns True if a input contains all zeros, else returns True.

# References
[0] <a id="toa-wikipedia"></a>https://en.wikipedia.org/wiki/Theory_of_computation<br>
[1] https://www.youtube.com/watch?v=0D7yInuKvKs<br>
[2] <a id="03-computation"></a>https://github.com/ianmcloughlin/2223-S1-emerging-technologies/blob/main/notebooks/03-computation.ipynb<br>